<a href="https://colab.research.google.com/github/zongming03/Feature-Engineering/blob/main/IndividualAssignment_WIE3007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install featuretools

In [25]:
import featuretools as ft
import pandas as pd

##Load Dataset

In [26]:
# Create DataFrames for customers, products, orders, and order details

customers = {
    "CustomerID": [101, 102, 103, 104, 105],
    "Name": ['John Doe', 'Jane Smith', 'Mike Jordan', 'Emma Stone', 'Chris Evans'],
    "Email": ['john.doe@example.com', 'jane.smith@example.com', 'mike.jordan@example.com', 'emma.stone@example.com', 'chris.evans@example.com'],
    "SignupDate": ['2023-01-10', '2023-01-15', '2023-01-20', '2023-02-01', '2023-02-05']
}

products = {
    'ProductID': [201, 202, 203, 204, 205],
    'Name': ['Laptop', 'Tablet', 'Smartphone', 'Headphones', 'Smartwatch'],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Accessories', 'Electronics'],
    'Price': [1000, 500, 800, 200, 300]
}

orders = {
    "OrderID": [301, 302, 303, 304, 305, 306, 307],
    "CustomerID": [101, 102, 103, 104, 105, 101, 102],
    "OrderDate": ['2023-02-01', '2023-02-05', '2023-02-10', '2023-02-12', '2023-02-15', '2023-02-20', '2023-02-22'],
    "ShipDate": ['2023-02-03', '2023-02-07', '2023-02-12', '2023-02-14', '2023-02-17', '2023-02-22', '2023-02-25']
}

order_details = {
    "OrderID": [301, 302, 303, 304, 305, 306, 307, 304, 306, 307],
    "ProductID": [201, 202, 203, 204, 205, 201, 202, 203, 204, 205],
    "Quantity": [1, 2, 1, 1, 3, 2, 1, 1, 2, 1],
    "Discount": [0, 0.1, 0, 0.05, 0.15, 0, 0.1, 0.05, 0, 0.1]
}

customers_df = pd.DataFrame(customers)
products_df = pd.DataFrame(products)
orders_df = pd.DataFrame(orders)
order_details_df = pd.DataFrame(order_details)

# Adding a combined key in the order details
order_details_df['OrderDetailsID'] = order_details_df['OrderID'].astype(str) + '_' + order_details_df['ProductID'].astype(str)


In [27]:
# Create new data
new_customers = {
    "CustomerID": [106, 107, 108, 109, 110],
    "Name": ['Alice Brown', 'Bob Johnson', 'Charlie Davis', 'David Garcia', 'Emily Wilson'],
    "Email": ['alice.brown@example.com', 'bob.johnson@example.com', 'charlie.davis@example.com', 'david.garcia@example.com', 'emily.wilson@example.com'],
    "SignupDate": ['2023-02-10', '2023-02-15', '2023-02-20', '2023-03-01', '2023-03-05']
}

# Concatenate with existing DataFrame
customers_df = pd.concat([customers_df, pd.DataFrame(new_customers)], ignore_index=True)

# Create new data
new_products = {
    'ProductID': [206, 207, 208, 209, 210],
    'Name': ['Mouse', 'Keyboard', 'Monitor', 'Printer', 'Webcam'],
    'Category': ['Accessories', 'Accessories', 'Electronics', 'Electronics', 'Accessories'],
    'Price': [50, 100, 300, 250, 150]
}

# Concatenate with existing DataFrame
products_df = pd.concat([products_df, pd.DataFrame(new_products)], ignore_index=True)

# Create new data
new_orders = {
    "OrderID": [308, 309, 310, 311, 312],
    "CustomerID": [103, 104, 105, 101, 102],
    "OrderDate": ['2023-03-01', '2023-03-05', '2023-03-10', '2023-03-12', '2023-03-15'],
    "ShipDate": ['2023-03-03', '2023-03-07', '2023-03-12', '2023-03-14', '2023-03-17']
}

# Concatenate with existing DataFrame
orders_df = pd.concat([orders_df, pd.DataFrame(new_orders)], ignore_index=True)

# Create new data
new_order_details = {
    "OrderID": [308, 309, 310, 311, 312, 308, 309, 310],
    "ProductID": [204, 205, 206, 207, 208, 201, 202, 203],
    "Quantity": [1, 2, 1, 1, 3, 2, 1, 1],
    "Discount": [0, 0.1, 0, 0.05, 0.15, 0, 0.1, 0.05]
}

# Concatenate with existing DataFrame
order_details_df = pd.concat([order_details_df, pd.DataFrame(new_order_details)], ignore_index=True)

# Update OrderDetailsID
order_details_df['OrderDetailsID'] = order_details_df['OrderID'].astype(str) + '_' + order_details_df['ProductID'].astype(str)

In [28]:
# Check for missing values in each DataFrame
print(customers_df.isnull().sum())
print(products_df.isnull().sum())
print(orders_df.isnull().sum())
print(order_details_df.isnull().sum())

# Check the data types
print(customers_df.dtypes)
print(products_df.dtypes)
print(orders_df.dtypes)
print(order_details_df.dtypes)

CustomerID    0
Name          0
Email         0
SignupDate    0
dtype: int64
ProductID    0
Name         0
Category     0
Price        0
dtype: int64
OrderID       0
CustomerID    0
OrderDate     0
ShipDate      0
dtype: int64
OrderID           0
ProductID         0
Quantity          0
Discount          0
OrderDetailsID    0
dtype: int64
CustomerID     int64
Name          object
Email         object
SignupDate    object
dtype: object
ProductID     int64
Name         object
Category     object
Price         int64
dtype: object
OrderID        int64
CustomerID     int64
OrderDate     object
ShipDate      object
dtype: object
OrderID             int64
ProductID           int64
Quantity            int64
Discount          float64
OrderDetailsID     object
dtype: object


## Entity Set

In [29]:
# creating entity set 'entitySet'
entitySet = ft.EntitySet(id="e-commerce_es")

#Add DataFrame to the EntitySet
#Customers
customers_entity = entitySet.add_dataframe(
    dataframe=customers_df,  #  Dataframe containing the data.
    dataframe_name='Customers',  # Unique name to associate with this dataframe
    index='CustomerID',  # Unique identifier
    time_index='SignupDate'
)

#Products
products_entity = entitySet.add_dataframe(
    dataframe=products_df,
    dataframe_name='Products',
    index='ProductID'
)

#Orders
orders_entity = entitySet.add_dataframe(
    dataframe=orders_df,
    dataframe_name='Orders',
    index='OrderID' ,
    time_index='OrderDate',
)

#OrdersDetails
orderDetails_entity = entitySet.add_dataframe(
    dataframe=order_details_df,
    dataframe_name='OrderDetails',
    index='OrderDetailsID'
)

/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

####Establish relationship

In [30]:
# Define relationships
entitySet.add_relationship(parent_dataframe_name='Customers', parent_column_name='CustomerID',
                    child_dataframe_name='Orders', child_column_name='CustomerID')

entitySet.add_relationship(parent_dataframe_name='Products', parent_column_name='ProductID',
                    child_dataframe_name='OrderDetails', child_column_name='ProductID')

entitySet.add_relationship(parent_dataframe_name='Orders', parent_column_name='OrderID',
                    child_dataframe_name='OrderDetails', child_column_name='OrderID')

Entityset: e-commerce_es
  DataFrames:
    Customers [Rows: 10, Columns: 4]
    Products [Rows: 10, Columns: 4]
    Orders [Rows: 12, Columns: 4]
    OrderDetails [Rows: 18, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrderDetails.ProductID -> Products.ProductID
    OrderDetails.OrderID -> Orders.OrderID

####check entity

In [31]:
print(entitySet)

Entityset: e-commerce_es
  DataFrames:
    Customers [Rows: 10, Columns: 4]
    Products [Rows: 10, Columns: 4]
    Orders [Rows: 12, Columns: 4]
    OrderDetails [Rows: 18, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrderDetails.ProductID -> Products.ProductID
    OrderDetails.OrderID -> Orders.OrderID


#Deep Feature Synthesis (DFS)

In [32]:
# Generate features with DFS for Orders fact table
feature_matrix, feature_defs = ft.dfs(
    entityset=entitySet,
    target_dataframe_name="Orders",
    verbose=True,
    max_depth=3,
    agg_primitives=['sum', 'mean', 'count', 'max', 'min'],
    trans_primitives=['month', 'year', 'weekday']
)


Built 95 features
Elapsed: 00:00 | Progress:   0%|          

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7d405edf4700> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x7d405edf5000> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x7d405edf4820> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable wil

Elapsed: 00:00 | Progress:  48%|████▊     

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x7d405edf5000> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7d405edf40d0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7d405edf4700> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable wil

Elapsed: 00:00 | Progress: 100%|██████████


In [33]:
# PRINT THE GENERATED FEATURE MATRIX
print(feature_matrix.head())
feature_matrix.columns

         CustomerID  COUNT(OrderDetails)  MAX(OrderDetails.Discount)  \
OrderID                                                                
301             101                    1                        0.00   
302             102                    1                        0.10   
303             103                    1                        0.00   
304             104                    2                        0.05   
305             105                    1                        0.15   

         MAX(OrderDetails.Quantity)  MEAN(OrderDetails.Discount)  \
OrderID                                                            
301                             1.0                         0.00   
302                             2.0                         0.10   
303                             1.0                         0.00   
304                             1.0                         0.05   
305                             3.0                         0.15   

         MEAN(Orde

Index(['CustomerID', 'COUNT(OrderDetails)', 'MAX(OrderDetails.Discount)',
       'MAX(OrderDetails.Quantity)', 'MEAN(OrderDetails.Discount)',
       'MEAN(OrderDetails.Quantity)', 'MIN(OrderDetails.Discount)',
       'MIN(OrderDetails.Quantity)', 'SUM(OrderDetails.Discount)',
       'SUM(OrderDetails.Quantity)', 'MONTH(OrderDate)', 'MONTH(ShipDate)',
       'WEEKDAY(OrderDate)', 'WEEKDAY(ShipDate)', 'YEAR(OrderDate)',
       'YEAR(ShipDate)', 'MAX(OrderDetails.Products.Price)',
       'MEAN(OrderDetails.Products.Price)', 'MIN(OrderDetails.Products.Price)',
       'SUM(OrderDetails.Products.Price)', 'Customers.COUNT(Orders)',
       'Customers.COUNT(OrderDetails)', 'Customers.MAX(OrderDetails.Discount)',
       'Customers.MAX(OrderDetails.Quantity)',
       'Customers.MEAN(OrderDetails.Discount)',
       'Customers.MEAN(OrderDetails.Quantity)',
       'Customers.MIN(OrderDetails.Discount)',
       'Customers.MIN(OrderDetails.Quantity)',
       'Customers.SUM(OrderDetails.Discount)',
   

In [34]:
feature_matrix.to_csv('feature_matrix.csv', index=False)